In [1]:
s3_prediction = "s3://aegovan-data/chemprot/predictions_chemprot-bert-f1-2022-05-22-23-18-21-560/prediction_chemprot_test.json"

In [2]:
import sys, os, pandas as pd
sys.path.append("../src")

import utils

In [3]:
from utils.s3_utils import download_file as download_s3_file


In [4]:
def write_chemprot_eval(df, dest_file):
    df = df[["abstract_id","prediction", "participant1_id", "participant2_id"]].query("prediction != 'NEGATIVE'").copy(deep=True)
    

    df["participant1_id"] = df["participant1_id"].apply(lambda x: f"Arg1:{x}")
    df["participant2_id"] = df["participant2_id"].apply(lambda x: f"Arg2:{x}")
    df.to_csv(dest_file, sep="\t", header=False, index=False)

In [5]:
local_temp ="./temp"

In [6]:
local_prediction_dir = os.path.join(local_temp, "prediction")
os.makedirs(local_prediction_dir, exist_ok=True)
local_prediction_file  = download_s3_file( s3_prediction, local_prediction_dir)

In [7]:
df_pred = pd.read_json(local_prediction_file)
df_pred.head()

,abstract_id,abstract,sentence_raw,sentence_anonymised,participant1_id,participant1,participant2_id,participant2,relationship_type,relationship_group,...,NEGATIVE,CPR:3,CPR:4,CPR:5,CPR:6,CPR:9,prediction,confidence,confidence_std,raw_confidence
0,10076535,Androgen antagonistic effect of estramustine p...,Binding of hydroxyflutamide to m-AR in LNCaP c...,Binding of _CHEMICAL_ to m- _GENE-Y_ in LNCa...,T23,"{'abstract_id': '10076535', 'id': 'T23', 'enti...",T55,"{'abstract_id': '10076535', 'id': 'T55', 'enti...",DIRECT-REGULATOR,CPR:2,...,0.997153,0.000678,0.000400,0.000484,0.000466,0.000820,NEGATIVE,0.997153,NaN,[0.997152805328369]
1,10076535,Androgen antagonistic effect of estramustine p...,"Analogous to the antiandrogens, bicalutamide a...","Analogous to the antiandrogens, _CHEMICAL_ a...",T2,"{'abstract_id': '10076535', 'id': 'T2', 'entit...",T48,"{'abstract_id': '10076535', 'id': 'T48', 'enti...",DIRECT-REGULATOR,CPR:2,...,0.996619,0.000680,0.000724,0.000231,0.000899,0.000847,NEGATIVE,0.996619,NaN,[0.9966193437576291]
2,10076535,Androgen antagonistic effect of estramustine p...,"Analogous to the antiandrogens, bicalutamide a...","Analogous to the antiandrogens, bicalutamide a...",T3,"{'abstract_id': '10076535', 'id': 'T3', 'entit...",T48,"{'abstract_id': '10076535', 'id': 'T48', 'enti...",DIRECT-REGULATOR,CPR:2,...,0.997097,0.000590,0.000459,0.000309,0.000799,0.000746,NEGATIVE,0.997097,NaN,[0.997097015380859]
3,10076535,Androgen antagonistic effect of estramustine p...,"Analogous to the antiandrogens, bicalutamide a...","Analogous to the antiandrogens, bicalutamide a...",T4,"{'abstract_id': '10076535', 'id': 'T4', 'entit...",T48,"{'abstract_id': '10076535', 'id': 'T48', 'enti...",DIRECT-REGULATOR,CPR:2,...,0.994657,0.000585,0.000948,0.000264,0.000819,0.002728,NEGATIVE,0.994657,NaN,[0.9946566820144651]
4,10076535,Androgen antagonistic effect of estramustine p...,Binding of hydroxyflutamide to m-AR in LNCaP c...,Binding of _CHEMICAL_ to m-AR in LNCaP cells...,T23,"{'abstract_id': '10076535', 'id': 'T23', 'enti...",T56,"{'abstract_id': '10076535', 'id': 'T56', 'enti...",INDIRECT-UPREGULATOR,CPR:3,...,0.239240,0.752279,0.002164,0.003034,0.001404,0.001880,CPR:3,0.752279,NaN,[0.7522785663604731]


In [8]:
write_chemprot_eval(df_pred, os.path.join(local_temp, f"eval_pred_{local_prediction_file.split(os.path.sep)[-1]}.tsv"))

In [12]:
df_pred.query("abstract_id == '10351938'")[["prediction", "participant1_id", "participant2_id"]].sort_values(by=["prediction", "participant1_id"])

,prediction,participant1_id,participant2_id
32,CPR:3,T11,T33
33,CPR:3,T12,T32
34,CPR:3,T12,T33
36,CPR:3,T13,T33
40,CPR:3,T15,T33
29,CPR:3,T20,T34
30,CPR:3,T21,T34
41,CPR:3,T8,T28
42,CPR:4,T23,T34
43,CPR:4,T25,T36


In [10]:
pd.read_json("../chemprot_test.json").query("abstract_id == '10076535'")[["participant1_id", "participant2_id"]]

,participant1_id,participant2_id
0,T23,T55
1,T2,T48
2,T3,T48
3,T4,T48
4,T23,T56
5,T18,T49
6,T18,T50
7,T18,T51
8,T19,T49
9,T19,T50
